# CoAP ve Python

Onur Güzel  
Artistanbul  

---

Python Saati #26  
10 Ekim 2017, Hipo, İstanbul

## Ne Anlatacağım?

- CoAP Nedir? (Yenir mi?)

- Python ile CoAP

- IKEA? 🤔

## CoAP Nedir?

- "Constrained Application Protocol"  
Bir IETF CoRE grubu standardı (RFC 7252)

- IoT için REST: GET, POST, PUT, DELETE

- HTTP Proxy  
http<i></i>://example.com/hc/**coap://c.example.com/obj**

- UDP ve DTLS

## CoAP ve yardımcı standartlar

- OMA Lightweight M2M: [openmobilealliance.org](http://openmobilealliance.org/iot/lightweight-m2m-lwm2m)

- Anahtar kelime: Birlikte çalıştırılabilirlik

- IPSO Smart Objects: [ipso-alliance.github.io/pub](https://ipso-alliance.github.io/pub/)

## Python ile CoAP

- CoAPthon: [github.com/Tanganelli/CoAPthon](https://github.com/Tanganelli/CoAPthon)

 `pip install CoAPthon`

- aiocoap: [github.com/chrysn/aiocoap](https://github.com/chrysn/aiocoap)

 `pip install aiocoap`

# CoAPthon (Python 2.7 😒)

```
from coapthon.client.helperclient import HelperClient

client = HelperClient(server=("192.168.1.30", 5683))
response = client.get("/.well-known/core")
print(response.pretty_print())
client.stop()
```

☝️ Yukarıdaki kod bloğu, CoAPthon modülü Python 3 desteklemediği için Markdown olarak yazıldı.

# aiocoap

In [ ]:
import asyncio
import credentials
from aiocoap.transports import tinydtls

class MyDTLSSecurityStore:
    SECRET_PSK = credentials.security_code.encode('utf-8')
    def _get_psk(self, host, port):
        return b"Client_identity", MyDTLSSecurityStore.SECRET_PSK

def _patched_datagram_received(self, data, addr):
    self.parent._dtls_socket.handleMessage(self.parent._connection, data, 0)

tinydtls.DTLSSecurityStore = MyDTLSSecurityStore
tinydtls.DTLSClientConnection.SingleConnection.datagram_received = \
    _patched_datagram_received
    
loop = asyncio.get_event_loop()

In [ ]:
import json
from aiocoap import Message, Context
from aiocoap.numbers.codes import Code

async def get_response(url, code=Code.GET, payload=None):
    protocol = await Context.create_client_context()
    kwargs = {}
    if payload is not None:
        kwargs['payload'] = json.dumps(payload).encode('utf-8')
    msg = Message(code=code, uri=f'coaps://192.168.1.14{url}', **kwargs)
    response = await protocol.request(msg).response
    return response

response = loop.run_until_complete(get_response('/.well-known/core'))
response.payload

In [ ]:
response = loop.run_until_complete(get_response('/15001/65537'))
response.payload

In [ ]:
response = loop.run_until_complete(get_response('/15001/65537', code=Code.PUT, payload={
    "3311":[
        {"5850":0}
    ]
}))
response.payload

## IKEA TRÅDFRI 💸

- pytradfri: [github.com/ggravlingen/pytradfri](https://github.com/ggravlingen/pytradfri)

 `pip install pytradfri`

- ve elle derlenen bazı bağımlılıklar 😢

In [ ]:
from pytradfri import Gateway
from pytradfri.api.libcoap_api import api_factory

api = api_factory('192.168.1.14', credentials.security_code)

gateway = Gateway()

devices_command = gateway.get_devices()
devices_commands = api(devices_command)
devices = api(*devices_commands)

light = [dev for dev in devices if dev.has_light_control][1]

# Teşekkürler

🤗